### **Experiment on Privacy-Preserving Techniques for Sensitive Data Matching in SPL Scenarios**

**Overview**

In an increasingly connected world, organizations often need to compare sensitive data against external lists, such as Suspicious Persons Lists (SPL), without compromising privacy. Privacy-preserving techniques such as Homomorphic Encryption (HE), Secure Multiparty Computation (SMPC), Differential Privacy (DP), and Hash-based Matching enable entities to collaborate securely on sensitive data without revealing the actual data. This experiment explores the effectiveness of these techniques in ensuring privacy and trust, particularly when public and private entities must collaborate.
As the demand for data privacy increases, particularly in contexts like national security, fraud prevention, and financial crime investigations, it's critical to strike a balance between data sharing and safeguarding individuals’ sensitive information.





### **Scenario (Use Case)**
**Suspicious Person List (SPL) Comparison**

The use case involves comparing publicly held SPLs, which include various unique identifiers (passport numbers, driver licenses, or national IDs), with customer lists held by private entities. This process is crucial for identifying potential threats while ensuring the privacy of both private companies and individuals.

**Privacy Considerations**

•	High Trust Situations: Data is encrypted in transit and at rest, with operations performed on secure systems.

•	Medium Trust Situations: Data is compared while encrypted, using privacy-preserving technologies, and governance arrangements are necessary between parties.

•	Low Trust Situations: Data is encrypted throughout the process, with results accessible only to the requesting public entity. Private parties do not learn the outcome unless further consent is obtained.

**Use Case Example: Public to Private – Low Trust**

In this scenario, public entities push SPL data to private entities for matching against their customer lists. The results are only accessible to the public party, ensuring that private entities do not gain any insights. This setup requires strong encryption methods, including homomorphic encryption and differential privacy, to maintain the confidentiality of data throughout the process.

Technology Involved:
1.	Homomorphic Encryption (HE): Enables encrypted data comparison without revealing underlying data.
2.	Symmetric Encryption: Ensures results are securely shared after matching.
3.	Differential Privacy (DP): Adds noise to ensure no individual data point can be identified in the results.
4.	Secure Multiparty Computation (SMPC): Allows multiple parties to jointly compute the result without revealing their inputs to each other.


### **Experimental Setup**

**Data Generation**

Synthetic data was generated to simulate the matching process between bank customers and suspicious persons:

•	Bank Customers: 5,000 records, each with a unique TFN, name, date of birth, and address.

•	Suspicious Persons: 50 records, similarly structured, with a 10% overlap in TFNs to simulate realistic matching scenarios.

**Privacy-Preserving Techniques**
1.	Hash Matching: TFNs are hashed and compared, simulating encryption.
2.	Homomorphic Encryption (HE) Matching: TFNs are encrypted using HE, and matches are identified by comparing encrypted values.
3.	Secure Multiparty Computation (SMPC) Matching: TFNs are secret-shared between parties, and matches are determined based on shared secrets.
4.	Differential Privacy (DP) Matching: Laplace noise is added to the match count to preserve privacy.

**Performance Metrics**

The performance of each method was measured based on:

•	Number of Matches Identified: How many customer records matched with suspicious persons.

•	Execution Time: The time taken to perform the matching process.



**1. Install Required Libraries**

In [ ]:
!pip install faker tenseal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 35.7 MB/s eta 0:00:00


**2. Hash Matching**


In [ ]:
import pandas as pd
import numpy as np
from faker import Faker
import hashlib
import tenseal as ts
import time
import random

# Function to generate synthetic data with 10% matching TFNs
def generate_synthetic_data(num_customers, num_suspicious):
    fake = Faker()

    # Generate unique TFNs for customers and suspicious persons
    tfn_customers = np.random.randint(0, 100000000, num_customers)
    tfn_suspicious = np.random.randint(0, 100000000, num_suspicious)

    # Ensure 10% overlap in TFNs between the two datasets
    overlap_count = int(0.1 * min(num_customers, num_suspicious))
    if overlap_count > 0:
        overlap_tfn = np.random.choice(tfn_customers, overlap_count, replace=False)
        tfn_suspicious[:overlap_count] = overlap_tfn

    # Shuffle TFNs to mix overlap
    np.random.shuffle(tfn_suspicious)

    # Generate customer data
    bank_customers = pd.DataFrame({
        'Customer_ID': np.arange(1, num_customers + 1),
        'Name': [fake.name() for _ in range(num_customers)],
        'Date_of_Birth': [fake.date_of_birth(minimum_age=18, maximum_age=90) for _ in range(num_customers)],
        'Address': [fake.address().replace("\n", ", ") for _ in range(num_customers)],
        'TFN': tfn_customers
    })

    # Generate suspicious persons data
    suspicious_persons = pd.DataFrame({
        'Suspicious_Person_ID': np.arange(1, num_suspicious + 1),
        'Name': [fake.name() for _ in range(num_suspicious)],
        'Date_of_Birth': [fake.date_of_birth(minimum_age=18, maximum_age=90) for _ in range(num_suspicious)],
        'Address': [fake.address().replace("\n", ", ") for _ in range(num_suspicious)],
        'TFN': tfn_suspicious
    })

    return bank_customers, suspicious_persons

# Function to simulate encryption by hashing the TFN
def simulate_encryption(tfn):
    return hashlib.sha256(str(tfn).encode()).hexdigest()

# Function to perform Hash Matching
def perform_hash_matching(bank_customers, suspicious_persons):
    # Encrypt the TFNs of bank customers
    encrypted_customers = bank_customers['TFN'].apply(simulate_encryption)

    # Encrypt the TFNs of suspicious persons
    encrypted_suspicious = suspicious_persons['TFN'].apply(simulate_encryption)

    # Perform matching (in encrypted domain)
    matches = []
    for enc_suspicious_tfn in encrypted_suspicious:
        match_indices = encrypted_customers[encrypted_customers == enc_suspicious_tfn].index
        matches.extend(bank_customers.loc[match_indices, 'Customer_ID'].tolist())

    return matches

# Function to perform Homomorphic Encryption Matching
def perform_he_matching(bank_customers, suspicious_persons):
    # Initialize TenSEAL context with BFV scheme
    context = ts.context(
        ts.SCHEME_TYPE.BFV,
        poly_modulus_degree=8192,
        plain_modulus=1032193
    )
    context.generate_galois_keys()
    context.generate_relin_keys()

    # Encrypt the TFNs of bank customers
    encrypted_customers = [ts.bfv_vector(context, [tfn]) for tfn in bank_customers['TFN']]

    # Encrypt the TFNs of suspicious persons
    encrypted_suspicious = [ts.bfv_vector(context, [tfn]) for tfn in suspicious_persons['TFN']]

    # Perform matching (in encrypted domain)
    matches = []
    for i, enc_suspicious_tfn in enumerate(encrypted_suspicious):
        for j, enc_customer_tfn in enumerate(encrypted_customers):
            diff = enc_suspicious_tfn - enc_customer_tfn
            decrypted_diff = diff.decrypt()[0]

            if decrypted_diff == 0:
                matches.append(bank_customers.iloc[j]['Customer_ID'])
                break

    return matches

# Function to simulate secret sharing
def secret_share(value):
    share1 = np.random.randint(0, 100000000)
    share2 = (value - share1) % 100000000
    return share1, share2

# Function to perform SMPC Matching
def perform_smpc_matching(bank_customers, suspicious_persons):
    customer_shares = {
        row['Customer_ID']: secret_share(row['TFN']) for _, row in bank_customers.iterrows()
    }
    suspicious_shares = {
        row['Suspicious_Person_ID']: secret_share(row['TFN']) for _, row in suspicious_persons.iterrows()
    }

    matches = []
    for suspicious_id, (s1, s2) in suspicious_shares.items():
        for customer_id, (c1, c2) in customer_shares.items():
            if (s1 + c2) % 100000000 == (s2 + c1) % 100000000:
                matches.append(customer_id)
                break  # Exit inner loop once a match is found

    return matches

# Function to add Laplace noise for differential privacy
def add_laplace_noise(value, epsilon):
    noise = np.random.laplace(0, 1/epsilon)
    return value + noise

# Function to perform Differential Privacy Matching
def perform_dp_matching(bank_customers, suspicious_persons, epsilon):
    matches = 0
    for tfn_suspicious in suspicious_persons['TFN']:
        if tfn_suspicious in bank_customers['TFN'].values:
            matches += 1

    private_matches = add_laplace_noise(matches, epsilon)
    return private_matches

# Main code execution
try:
    # User input for number of customers, suspicious persons, and epsilon
    num_customers = int(input("Enter the number of bank customers (up to 10000): "))
    num_suspicious = int(input("Enter the number of suspicious persons (up to 1000): "))
    epsilon = float(input("Enter the privacy budget (epsilon, e.g., 0.1 for DP method): "))

    # Input validation
    if num_customers > 10000 or num_customers <= 0:
        raise ValueError("The number of bank customers must be between 1 and 10000.")
    if num_suspicious > 1000 or num_suspicious <= 0:
        raise ValueError("The number of suspicious persons must be between 1 and 1000.")

    # Generate synthetic data
    bank_customers, suspicious_persons = generate_synthetic_data(num_customers, num_suspicious)

    # Perform and time Hash Matching
    start_time = time.time()
    matches_hash = perform_hash_matching(bank_customers, suspicious_persons)
    time_hash = time.time() - start_time

    # Perform and time Homomorphic Encryption Matching
    start_time = time.time()
    matches_he = perform_he_matching(bank_customers, suspicious_persons)
    time_he = time.time() - start_time

    # Perform and time SMPC Matching
    start_time = time.time()
    matches_smpc = perform_smpc_matching(bank_customers, suspicious_persons)
    time_smpc = time.time() - start_time

    # Perform and time Differential Privacy Matching
    start_time = time.time()
    matches_dp = perform_dp_matching(bank_customers, suspicious_persons, epsilon)
    time_dp = time.time() - start_time

    # Output results
    print(f"\nNumber of matches (Hash Matching): {len(matches_hash)}")
    print(f"Time taken (Hash Matching): {time_hash:.4f} seconds")

    print(f"\nNumber of matches (Homomorphic Encryption Matching): {len(matches_he)}")
    print(f"Time taken (HE Matching): {time_he:.4f} seconds")

    print(f"\nNumber of matches (SMPC Matching): {len(matches_smpc)}")
    print(f"Time taken (SMPC Matching): {time_smpc:.4f} seconds")

    print(f"\nEstimated number of matches (Differential Privacy Matching): {matches_dp:.2f}")
    print(f"Time taken (DP Matching): {time_dp:.4f} seconds")

except ValueError as e:
    print(e)
except Exception as e:
    print(f"An error occurred: {e}")


Enter the number of bank customers (up to 10000): 5000
Enter the number of suspicious persons (up to 1000): 50
Enter the privacy budget (epsilon, e.g., 0.1 for DP method): 0.1

Number of matches (Hash Matching): 5
Time taken (Hash Matching): 0.0418 seconds

Number of matches (Homomorphic Encryption Matching): 5
Time taken (HE Matching): 560.1197 seconds

Number of matches (SMPC Matching): 0
Time taken (SMPC Matching): 0.2970 seconds

Estimated number of matches (Differential Privacy Matching): 11.50
Time taken (DP Matching): 0.0006 seconds
